In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib.parse

In [ ]:
url = 'https://www.centris.ca/en/properties~for-sale'
headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62'})

In [ ]:
response = requests.get(url, headers=headers)
print(response)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())

In [ ]:
result = soup.find_all('div', {'class': 'shell'})
len(result)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
import json

headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62'})

with requests.Session() as session:
    session.headers.update(headers)
    url = f'https://www.centris.ca/en/houses~for-sale?view=Thumbnail&uc=1'
    # Add code to capture and include cookies if needed
    r = session.get(url)
    #return r.status_code
    soup = BeautifulSoup(r.content,'html.parser')

json_data = {"startPosition": 0}

In [ ]:

divs = soup.find_all('div', class_ = "shell")

house_list = []
for item in divs:

    title = item.find('meta')['content']
    print(title)

    raw_address = item.find('span', class_ = 'address').get_text()
    address = raw_address.strip() if raw_address else 'N/A'
    print(address)

    #house_list.append(title)

    link = item.find('a', class_ = 'a-more-detail')['href']
    print(link)

    price = item.find('meta', itemprop = 'price')['content']
    print(f'Price: {price}')

    sku = item.find('meta', itemprop = 'sku')['content']
    print(f'Sku: {sku}')

    bedrooms = item.find('div', class_ = "cac").get_text()
    print(f'Bedrooms: {bedrooms}')

    bathrooms = item.find('div', class_ = "sdb").get_text()
    print(f'Bathrooms: {bathrooms}')

    try:
        new_prop = item.find('div', class_ = "banner new-property")
        if len(new_prop) > 0:
            print('New Property: Yes')
        else:
            print('New Property: No')
    except:
         print('New Property: No')

    try:
        new_price = item.find('div', class_ = "banner new-price")
        if len(new_price) > 0:
            print('New Price: Yes')
        else:
            print('New Price: No')
    except:
         print('New Price: No')

    print('\n\n')

   
#df = pd.DataFrame({'title': house_list})

In [5]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://www.centris.ca/Property/GetInscriptions"
headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62',
}
json_data = {"startPosition": 0}

with requests.session() as s:
    Centris = []
    # load cookies:
    s.get(
        "https://www.centris.ca/en/houses~for-sale?view=Thumbnail&uc=1",
        headers=headers,
    )
    for page in range(0, 1000, 20):  # <-- increase number of pages here
        json_data["startPosition"] = page

        data = s.post(url, headers=headers, json=json_data).json()
        soup = BeautifulSoup(data["d"]["Result"]["html"], "html.parser")

        for a in soup.select(".a-more-detail"):
            title = a.select_one(".category").get_text(strip=True)
            address = a.select_one(".address").get_text(strip=True, separator="\n")
            link = "https://www.centris.ca" + a["href"]
            price = a.select_one(".price").get_text(strip=True)
            

            Centris.append((title, address, link, price))


df = pd.DataFrame(Centris, columns={'Title': title, 'Address': address, 'Link': link, 'Price': price})

with pd.ExcelWriter('C:\Development\property-price-tracker\centris.xlsx') as writer:
    df.to_excel(writer, sheet_name= 'Centris', index= False)

print( 'Data Saved To excel' )

Data Saved To excel
